In [ ]:
!pip3 install pandas dash

     |████████████████████████████████| 7.3 MB 10.6 MB/s 
     |████████████████████████████████| 23.9 MB 14 kB/s 
     |████████████████████████████████| 357 kB 67.5 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=3d8074f0b1f2b43d9386ae0b120f089fa3fd10428266e6db4d538ebe024651d1
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=f356d80a43bbbdd0f40a25a6512b2ce313685dc31c94ba2e38b9d9140a164bba
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=70cd84ed637b803852fc0c1bfdd68b20b122896c6633142a4bbe88a51a8a1b1d
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aef

In [ ]:
!pip install jupyter-dash

In [1]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import dash
#import dash_html_components as html
from dash import html
#import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

# Read the airline data into pandas dataframe
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

In [2]:
# Create a dash application
#app = dash.Dash(__name__)
# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# Build dash app layout
app.layout = html.Div(children=[ html.H1('Flight Delay Time Statistics', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 30}),
                                html.Div(["Input Year: ", dcc.Input(id='input-year', value='2010', 
                                type='number', style={'height':'35px', 'font-size': 30}),], 
                                style={'font-size': 30}),
                                html.Br(),
                                html.Br(), 
                                # Segment 1
                                html.Div([
                                        html.Div(dcc.Graph(id='carrier-plot')),
                                        html.Div(dcc.Graph(id='weather-plot'))
                                ], style={'display': 'flex'}),
                                # Segment 2
                                html.Div([
                                        html.Div(dcc.Graph(id='nas-plot')),
                                        html.Div(dcc.Graph(id='security-plot'))
                                ], style={'display': 'flex'}),
                                # Segment 3
                                html.Div(dcc.Graph(id='late-plot'), style={'width':'65%'})
                                ])

In [3]:
""" Compute_info function description

This function takes in airline data and selected year as an input and performs computation for creating charts and plots.

Arguments:
    airline_data: Input airline data.
    entered_year: Input year for which computation needs to be performed.
    
Returns:
    Computed average dataframes for carrier delay, weather delay, NAS delay, security delay, and late aircraft delay.

"""
def compute_info(airline_data, entered_year):
    # Select data
    df =  airline_data[airline_data['Year']==int(entered_year)]
    # Compute delay averages
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late

In [4]:
"""Callback Function

Function that returns fugures using the provided input year.

Arguments:

    entered_year: Input year provided by the user.
    
Returns:

    List of figures computed using the provided helper function `compute_info`.
"""
# Callback decorator
@app.callback( [
               Output(component_id='carrier-plot', component_property='figure'),
               Output(component_id='weather-plot', component_property='figure'),
               Output(component_id='nas-plot', component_property='figure'),
               Output(component_id='security-plot', component_property='figure'),
               Output(component_id='late-plot', component_property='figure')
               ],
               Input(component_id='input-year', component_property='value'))
# Computation to callback function and return graph
def get_graph(entered_year):
    
    # Compute required information for creating graph from the data
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(airline_data, entered_year)
            
    # Line plot for carrier delay
    carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
    #carrier_fig = go.Figure(data=go.Scatter(x=avg_car['Month'], y=avg_car['CarrierDelay'], mode='lines'))  #only works with one color
    #carrier_fig.update_layout(title='Average carrier delay time (minutes) by airline', xaxis_title='Month', yaxis_title='CarrierDelay') 
    
    # Line plot for weather delay
    weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
    #weather_fig = go.Figure(data=go.Scatter(x=avg_weather['Month'], y=avg_weather['WeatherDelay'], mode='lines', marker=dict(color='green')))
    #weather_fig.update_layout(title='Average weather delay time (minutes) by airline', xaxis_title='Month', yaxis_title='WeatherDelay') 
    # Line plot for nas delay
    nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
    #nas_fig = go.Figure(data=go.Scatter(x=avg_NAS['Month'], y=avg_NAS['NASDelay'], mode='lines', marker=dict(color='green')))
    #nas_fig.update_layout(title='Average NAS delay time (minutes) by airline', xaxis_title='Month', yaxis_title='NASDelay') 
    # Line plot for security delay
    sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
    #sec_fig = go.Figure(data=go.Scatter(x=avg_sec['Month'], y=avg_sec['SecurityDelay'], mode='lines', marker=dict(color='green')))
    #sec_fig.update_layout(title='Average security delay time (minutes) by airline', xaxis_title='Month', yaxis_title='SecurityDelay')   
    # Line plot for late aircraft delay
    late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
    #late_fig = go.Figure(data=go.Scatter(x=avg_late['Month'], y=avg_late['LateAircraftDelay'], mode='lines', marker=dict(color='green')))
    #late_fig.update_layout(title='Average late aircraft delay time (minutes) by airline', xaxis_title='Month', yaxis_title='LateAircraftDelay')        
    return[carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]



# Run the app
if __name__ == '__main__':
    app.run_server()
   # app.run_server( host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False)

Dash app running on:


<IPython.core.display.Javascript object>